<a href="https://colab.research.google.com/github/SenorFoca/CMF/blob/main/CMF_CW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Option Pricing

In [ ]:
import yfinance as yf
import numpy as np
from scipy.stats import norm

In [ ]:
# Download stock data
data = yf.download('SHEL.L', start='2023-01-01', end='2024-11-30')

[*********************100%***********************]  1 of 1 completed


In [ ]:

# Calculate daily returns
data['Daily Return'] = data['Adj Close'].pct_change()

# Calculate rolling volatility (e.g., 30-day standard deviation)
data['Rolling Volatility (30-day)'] = data['Daily Return'].rolling(window=30).std()

# Calculate annualized volatility
data['Annualized Volatility'] = data['Rolling Volatility (30-day)'] * (252 ** 0.5)

# Select relevant columns for display
result = data[['Adj Close', 'Annualized Volatility']].dropna()

# Display the result
print(result)

Price         Adj Close Annualized Volatility
Ticker           SHEL.L                      
Date                                         
2023-02-14  2342.885742              0.216087
2023-02-15  2359.813721              0.185033
2023-02-16  2389.933594              0.185474
2023-02-17  2346.983887              0.192209
2023-02-20  2331.281982              0.192080
...                 ...                   ...
2024-11-25  2574.000000              0.203394
2024-11-26  2554.500000              0.178075
2024-11-27  2538.000000              0.176385
2024-11-28  2531.500000              0.174904
2024-11-29  2531.500000              0.173550

[454 rows x 2 columns]


In [ ]:
import numpy as np
from scipy.stats import norm

# Option parameters
S = 25.31  # Stock price
K = 29  # Strike price
r = 0.05  # Risk-free rate
T = 1  # Time to maturity in years
sigma = 0.17  # Volatility
n_simulations = 100000  # Number of simulations for Monte Carlo

# Black-Scholes-Merton Formula
def black_scholes_call_price(S, K, r, T, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    call_price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    return call_price

bsm_price = black_scholes_call_price(S, K, r, T, sigma)

# Monte Carlo Simulation
np.random.seed(42)  # For reproducibility
z = np.random.standard_normal(n_simulations)  # Standard normal random variables
S_T = S * np.exp((r - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * z)  # Simulated prices
payoffs = np.maximum(S_T - K, 0)  # Payoff of call option
mc_price = np.exp(-r * T) * np.mean(payoffs)  # Discounted payoff

print(f'Black-Scholes-Merton Formula {bsm_price: .5f}')
print(f'Monte Carlo Simulation {mc_price: .5f}')

Black-Scholes-Merton Formula  0.87797
Monte Carlo Simulation  0.88218


# 2. Implied volatility

In [ ]:
import numpy as np
from scipy.stats import norm

# =======================
# Black-Scholes Functions
# =======================
def d1(S, K, r, sigma, T):
    return (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))

def d2(S, K, r, sigma, T):
    return d1(S, K, r, sigma, T) - sigma*np.sqrt(T)

def call_price_bs(S, K, r, sigma, T):
    """
    Compute the Black-Scholes price of a European call option
    for non-dividend paying underlying.
    """
    D1 = d1(S, K, r, sigma, T)
    D2 = d2(S, K, r, sigma, T)
    return S*norm.cdf(D1) - K*np.exp(-r*T)*norm.cdf(D2)

def vega(S, K, r, sigma, T):
    """
    Vega is the derivative of the call price w.r.t. sigma.
    """
    D1 = d1(S, K, r, sigma, T)
    return S * norm.pdf(D1)*np.sqrt(T)


# ===============================
# Newton-Raphson for Implied Vol
# ===============================
def implied_vol_newton(c_market, S, K, r, T, tol=1e-8, max_iter=100, initial_guess=0.2):


    sigma = initial_guess
    for i in range(max_iter):
        # Compute price difference
        price = call_price_bs(S, K, r, sigma, T)
        diff = price - c_market

        if abs(diff) < tol:
            # Converged
            return sigma

        # Compute Vega
        v = vega(S, K, r, sigma, T)

        # Update sigma using Newton-Raphson
        sigma = sigma - diff / v

    # If we reach here, we did not converge within max_iter
    raise ValueError("Implied volatility did not converge.")


S = 100.0
K = 100.0
r = 0.05
T = 1.0
c_market = 10.0


implied_vol = implied_vol_newton(c_market, S, K, r, T, initial_guess=0.2)
print("Implied Volatility:", implied_vol)

Implied Volatility: 0.187971649526008


## Tesla Implied Volatility


In [ ]:
import numpy as np
import scipy.stats as si

def newton_vol_put(S, K, T, P, r):
    MAX_ITERATIONS = 100
    tolerance = 1e-8
    min_vega = 1e-10  # Threshold for considering vega too small
    sigma = 0.25  # initial guess

    for i in range(MAX_ITERATIONS):
        d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T)/(sigma*np.sqrt(T))
        d2 = d1 - sigma*np.sqrt(T)

        price = K*np.exp(-r*T)*si.norm.cdf(-d2) - S*si.norm.cdf(-d1)
        vega = S * np.sqrt(T) * si.norm.pdf(d1)

        diff = P - price

        if abs(diff) < tolerance:
            return sigma

        if vega < min_vega:
            print(f"Warning: Vega is too small (vega={vega}). The Newton-Raphson step may be unstable.")
            # Optionally break or return current sigma
            break

        sigma = sigma + diff/vega
        # Debug print statements (optional)
        # print(i, sigma, diff, vega)

    return sigma

In [ ]:
ticker = yf.Ticker("TSLA")
data = ticker.history(period="1d")  # last day’s data
current_price = data["Close"].iloc[-1]  # current spot price
print("Current TSLA spot price:", S)

Current TSLA spot price: 402.10089111328125


In [ ]:
def time_to_maturity(expiration_str, current_date=date.today()):

    expiration = datetime.strptime(expiration_str, "%Y-%m-%d").date()
    days_to_exp = (expiration - current_date).days
    T = days_to_exp / 365.0
    return T

In [ ]:
def get_put_price(ticker, expiration_str, strike):

    t = yf.Ticker(ticker)
    if expiration_str not in t.options:
        raise ValueError(f"Expiration date {expiration_str} not found for {ticker}.")

    puts = t.option_chain(expiration_str).puts
    put_row = puts[puts['strike'] == strike]
    if put_row.empty:
        raise ValueError(f"No put found for strike {strike} on {expiration_str} for {ticker}.")

    P = put_row['lastPrice'].values[0]
    return P

### K = 140

In [ ]:
ticker = "TSLA"
expiration_str = "2025-01-17"
strike = 140.0

S = current_price
K = strike
T = time_to_maturity(expiration_str)
P = get_put_price(ticker, expiration_str, strike)
r = 0.05

print("Spot price: ", S)
print("Put price: ", P)

Spot price:  402.260009765625
Put price:  0.19


In [ ]:
impvol = newton_vol_put(S, K, T, P, r)

In [ ]:
print("Estimated implied volatility in %:", round(impvol*100, 2), "%")

### K = 340


In [ ]:
ticker = "TSLA"
expiration_str = "2025-01-17"
strike = 340.0

S = current_price
K = strike
T = time_to_maturity(expiration_str)
P = get_put_price(ticker, expiration_str, strike)
r = 0.05

print("Spot price: ", S)
print("Put price: ", P)

Spot price:  402.260009765625
Put price:  7.1


In [ ]:
impvol = newton_vol_put(S, K, T, P, r)

In [ ]:
print("Estimated implied volatility in %:", round(impvol*100, 2), "%")

Estimated implied volatility in %: 59.49 %


# 3. Tesla Annual Historical Volatility

In [ ]:
TSLA = yf.download("TSLA", start="2023-11-25", end="2024-12-10")

In [ ]:
spot_price = TSLA['Adj Close'].iloc[-1]
S = spot_price.values[0]
print('The spot price is $', round(S,2), '.')

The spot price is $ 389.79 .


In [ ]:
log_return = np.log(TSLA['Adj Close'] / TSLA['Adj Close'].shift(1))
vol_h = np.sqrt(252) * log_return.std()
vol = vol_h.values[0]
print('The annualised volatility is', round(vol*100,2), '%')

The annualised volatility is 59.95 %


# 3. Binomial Tree

In [ ]:
import math
import numpy as np

In [ ]:
# Parameters
S0 = 100.0   # spot stock price
K = 100.0    # strike price
r = 0.05     # risk-free rate (annual)
sigma = 0.2  # volatility (annual)
T = 1.0      # time to maturity (years)
n = 3        # number of steps

dt = T / n                            # Delta t
u = math.exp(sigma * math.sqrt(dt))   # up factor
d = 1.0 / u                           # down factor
a = math.exp(r * dt)                  # risk free compound return
p = (a - d) / (u - d)                 # risk neutral up probability

print(f"u = {u:.5f}, d = {d:.5f}, p = {q:.5f}")

u = 1.12240, d = 0.89095, p = 0.54378


In [ ]:
# Binomial Tree
S = np.zeros(( n + 1, n + 1))

# Fill the binomial tree stock price matrix.
for k in range(n + 1):                  # Each column k represents a time step (from 0 to n).
    for j in range(k + 1):              # Each row j in column k represents the price after
                                        # k steps with exactly j down-moves
        S[j, k] = S0 * (u**(k - j)) * (d**j)  # Start from S0 and apply (k-j) up moves and j down moves.

print("Stock Price Matrix:")
print(S)

Stock Price Matrix:
[[100.         112.24009024 125.97837858 141.39824581]
 [  0.          89.09472523 100.         112.24009024]
 [  0.           0.          79.37870064  89.09472523]
 [  0.           0.           0.          70.72223522]]


In [ ]:
# At maturity, the option value equals intrinsic value: max(S-K, 0)
S_T = S[:,-1]
# Initialize the option value matrix V
V = np.zeros((n+1, n+1))
V[:,-1] = np.maximum(S_T-K, 0.0)    # Set the terminal (maturity) values of the option

# Backward induction for a European call option:
for j in range(n-1, -1, -1):
    for i in range(j+1):
        V[i,j] = ((p * V[i, j + 1] + (1 - p)*V[i + 1, j + 1])) / a

print("\nOption Price Matrix:")
print(V)

print(f"\nThe value of the American call option at time 0 is: {V[0,0]:.4f}")


Option Price Matrix:
[[11.04387109 17.71388824 27.6312332  41.39824581]
 [ 0.          3.50065379  6.54586268 12.24009024]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]]

The value of the American call option at time 0 is: 11.0439
